# Adding Nutrint Load to Initial particle positions and Pylag outputs

### Notebook Overview

This notebook processes nutrient flux data from stream watersheds and integrates it with particle tracking outputs.

---

#### 1. Read Zonal Statistics for Stream Watersheds

We begin by reading the zonal statistics output, which represents the nutrient flux (Total Nitrogen - **TN** and Total Phosphorus - **TP**) from each stream watershed to Lake Huron. These nutrient values are then associated with their corresponding `group_id` to facilitate linkage with particle tracking results.

---

#### 2. Integrate Nutrient Loads with Particle Tracking Outputs

Next, we add TN and TP values to the particle tracking dataset by:

- Counting the number of unique `group_id` entries (i.e., the number of particles released from each stream watershed).
- Dividing the total TN and TP values by the number of particles released **and** the number of unique days in each release month.

For example, for `release_month = 202301`:

$TN_{\text{gram per particle per day}} = \dfrac{Total N for the group_id}{\text{number of particles in Jan 2023}} *1000$

This allows for a per-particle representation of nutrient loads, supporting fine-scale spatial analysis of nutrient delivery to the lake.


# Imports

In [2]:
# some histogram analysis tools
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
#import seaborn as sns
# reas the first file from the file
import xarray as xr
import pandas as pd
import re
from collections import defaultdict
import glob

# Inputs

In [3]:
# read the initial points from a file
path = '/home/abolmaal/modelling/FVCOM/Huron/input/initial_position'
initial_positions_file = os.path.join(path, 'initial_positions_releasezone_intersection_multigroup_middleplume_final.dat')

# Output

In [35]:
out_path = '/home/abolmaal/modelling/FVCOM/Huron/InDirectNP_load/2024'

# Functions 

In [5]:
# read the initial_positions file
Initial_position = pd.read_csv(initial_positions_file, sep=' ',header=None, names=['group_id', 'lon', 'lat', 'depth'])
# drop the first row
Initial_position = Initial_position.drop(0, axis=0)
Initial_position.head()

,group_id,lon,lat,depth
1,0,275.3339,45.7428,0.0
2,0,275.3351,45.7428,0.0
3,0,275.3363,45.7428,0.0
4,0,275.3375,45.7428,0.0
5,0,275.3387,45.7428,0.0


In [6]:
# Step 1: Count occurrences of each unique group_id
# Step 2: Combine 'group_id' and 'group_number' with leading zeros
# Step 3 : Add a column call particle id  with Combine 'group_id' and 'group_number'
# Step 1: Add a particle index within each group_id
Initial_position['particle_index'] = Initial_position.groupby('group_id').cumcount()

# Step 2: Create particle_id as group_id (no padding) + particle_index (zero-padded to 2 digits)
Initial_position['particle_id'] = Initial_position.apply(
    lambda row: f"{int(row['group_id'])}{int(row['particle_index']):02d}", axis=1
)

In [7]:
# count the number of unique group_id in the intersection_df
unique_group_ids = Initial_position['group_id'].unique()
print(f"Number of unique group_ids: {len(unique_group_ids)}")

Number of unique group_ids: 114


In [8]:
# Reorder columns as requested
Initial_position = Initial_position[['group_id', 'particle_id', 'lon', 'lat', 'depth']]

Initial_position.head()

,group_id,particle_id,lon,lat,depth
1,0,000,275.3339,45.7428,0.0
2,0,001,275.3351,45.7428,0.0
3,0,002,275.3363,45.7428,0.0
4,0,003,275.3375,45.7428,0.0
5,0,004,275.3387,45.7428,0.0


# Adding TN TP from Zonal Stats to the initial file 

In [9]:
DirectNPcoast = pd.read_csv('/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/ZonalStats/DirectTNTP_StreamWatresheds.csv')

In [10]:
DirectNPcoast.columns

Index(['Group_id', 'Shape_Area', 'StreamDirectTP_kgday',
       'StreamDirectTN_kgday'],
      dtype='object')

In [11]:
# add two columns to the DirectNPcoast dataframe called 'StreamDirectTN_gramyr,' and 'StreamDirectTP_gramyr'
DirectNPcoast['StreamDirectTN_gramday'] = DirectNPcoast['StreamDirectTN_kgday'] * 1000 
DirectNPcoast['StreamDirectTP_gramday'] = DirectNPcoast['StreamDirectTP_kgday'] * 1000 
DirectNPcoast

,Group_id,Shape_Area,StreamDirectTP_kgday,StreamDirectTN_kgday,StreamDirectTN_gramday,StreamDirectTP_gramday
0,0,2.608193e+07,0.974675,13.480809,1.348081e+04,974.675289
1,1,6.915027e+07,2.000936,51.878556,5.187856e+04,2000.936025
2,2,1.703711e+08,4.179389,87.793043,8.779304e+04,4179.389247
3,3,9.425426e+07,3.598198,54.996792,5.499679e+04,3598.197912
4,4,3.772995e+07,1.326998,27.250568,2.725057e+04,1326.998050
...,...,...,...,...,...,...
140,170,1.961733e+07,4.299652,63.984811,6.398481e+04,4299.652109
141,171,1.228468e+03,0.000563,0.013980,1.398000e+01,0.563200
142,173,1.396735e+07,4.034916,23.636322,2.363632e+04,4034.915994
143,174,7.877722e+06,2.496025,14.078233,1.407823e+04,2496.024996


In [12]:
# in DirectNPcoast show which Group_ids has the min adn max values in the column 'DirectTP_StreamWatresheds' and DirectTN_kgday
min_directTP = DirectNPcoast['StreamDirectTP_kgday'].min()
max_directTP = DirectNPcoast['StreamDirectTP_kgday'].max()
min_directTN = DirectNPcoast['StreamDirectTN_kgday'].min()
max_directTN = DirectNPcoast['StreamDirectTN_kgday'].max()

min_directTP_group = DirectNPcoast[DirectNPcoast['StreamDirectTP_kgday'] == min_directTP]['Group_id']
max_directTP_group = DirectNPcoast[DirectNPcoast['StreamDirectTP_kgday'] == max_directTP]['Group_id']
min_directTN_group = DirectNPcoast[DirectNPcoast['StreamDirectTN_kgday'] == min_directTN]['Group_id']
max_directTN_group = DirectNPcoast[DirectNPcoast['StreamDirectTN_kgday'] == max_directTN]['Group_id']

print(f"Minimum DirectTP value: {min_directTP} in group(s): {min_directTP_group.values}")
print(f"Maximum DirectTP value: {max_directTP} in group(s): {max_directTP_group.values}")
print(f"Minimum DirectTN value: {min_directTN} in group(s): {min_directTN_group.values}")
print(f"Maximum DirectTN value: {max_directTN} in group(s): {max_directTN_group.values}")

Minimum DirectTP value: 0.0003174000012222 in group(s): [53]
Maximum DirectTP value: 996.7834509699624 in group(s): [162]
Minimum DirectTN value: 0.0058840001001954 in group(s): [53]
Maximum DirectTN value: 28752.26332026508 in group(s): [162]


In [13]:
# in DirectNPcoast show which Group_ids has the min adn max values in the column 'DirectTP_StreamWatresheds' and DirectTN_kgday
min_directTP = DirectNPcoast['StreamDirectTP_gramday'].min()
max_directTP = DirectNPcoast['StreamDirectTP_gramday'].max()
min_directTN = DirectNPcoast['StreamDirectTN_gramday'].min()
max_directTN = DirectNPcoast['StreamDirectTN_gramday'].max()
median_directTN = DirectNPcoast['StreamDirectTN_gramday'].median()
median_directTP = DirectNPcoast['StreamDirectTP_gramday'].median()
min_directTP_group = DirectNPcoast[DirectNPcoast['StreamDirectTP_gramday'] == min_directTP]['Group_id']
max_directTP_group = DirectNPcoast[DirectNPcoast['StreamDirectTP_gramday'] == max_directTP]['Group_id']
min_directTN_group = DirectNPcoast[DirectNPcoast['StreamDirectTN_gramday'] == min_directTN]['Group_id']
max_directTN_group = DirectNPcoast[DirectNPcoast['StreamDirectTN_gramday'] == max_directTN]['Group_id']
median_directTN_group = DirectNPcoast[DirectNPcoast['StreamDirectTN_gramday'] == median_directTN]['Group_id']
median_directTP_group = DirectNPcoast[DirectNPcoast['StreamDirectTP_gramday'] == median_directTP]['Group_id']
print(f"Minimum DirectTP value: {min_directTP} in group(s): {min_directTP_group.values}")
print(f"Maximum DirectTP value: {max_directTP} in group(s): {max_directTP_group.values}")
print(f"Minimum DirectTN value: {min_directTN} in group(s): {min_directTN_group.values}")
print(f"Maximum DirectTN value: {max_directTN} in group(s): {max_directTN_group.values}")
print(f"Median DirectTN value: {median_directTN} in group(s): {median_directTN_group.values}")
print(f"Median DirectTP value: {median_directTP} in group(s): {median_directTP_group.values}")

Minimum DirectTP value: 0.31740000122219997 in group(s): [53]
Maximum DirectTP value: 996783.4509699624 in group(s): [162]
Minimum DirectTN value: 5.8840001001954 in group(s): [53]
Maximum DirectTN value: 28752263.32026508 in group(s): [162]
Median DirectTN value: 39954.67197196558 in group(s): [87]
Median DirectTP value: 974.675289111927 in group(s): [0]


In [14]:
# count the nember of unique StWater_id in the data 
unique_stwater_ids = DirectNPcoast['Group_id'].nunique()
print(f"Number of unique Stream watersheds: {unique_stwater_ids}")

Number of unique Stream watersheds: 145


## Merging Direct NP to coast with Initial position

In [15]:
df = Initial_position
df1 = Initial_position
df2 = DirectNPcoast


In [16]:
def expand_and_merge_by_group_id_with_split_loads(df1, df2, group_col='group_id', split_cols=None):
    """
    Expands df2 rows based on group_id counts in df1, evenly distributes load values,
    and merges with df1.

    Parameters:
    - df1: DataFrame with many rows per group_id (e.g., particles)
    - df2: DataFrame with one row per group_id (e.g., watershed-level values)
    - group_col: Column name for group_id
    - split_cols: List of columns in df2 to distribute evenly across group_id count

    Returns:
    - merged_df: df1 with split load values and particle_id assigned
    """

    if split_cols is None:
        split_cols = ['StreamDirectTN_kgday', 'StreamDirectTP_kgday',
                      'StreamDirectTN_gramday', 'StreamDirectTP_gramday']

    # Standardize group_id column in df2
    df2 = df2.rename(columns={col: group_col for col in df2.columns if col.lower() == group_col.lower() or col.lower() == 'group_id'})

    # Step 1: Count group_id occurrences in df1
    group_counts = df1[group_col].value_counts().rename_axis(group_col).reset_index(name='count')

    # Step 2: Merge count into df2
    df2_expanded = df2.merge(group_counts, on=group_col, how='inner')

    # Step 3: Divide selected columns by count to spread evenly across particles
    for col in split_cols:
        if col in df2_expanded.columns:
            df2_expanded[col] = df2_expanded[col] / df2_expanded['count']

    # Step 4: Repeat df2 rows based on count
    df2_repeated = df2_expanded.loc[df2_expanded.index.repeat(df2_expanded['count'])].reset_index(drop=True)

    # Step 5: Align df1 to match repeated structure
    df1 = df1[df1[group_col].isin(df2_repeated[group_col])].reset_index(drop=True)
    df2_repeated = df2_repeated.drop(columns='count').reset_index(drop=True)

    # Add particle_index within each group_id
    df1['particle_index'] = df1.groupby(group_col).cumcount()

    # Create unique particle_id
    df1['particle_id'] = df1.apply(
        lambda row: f"{int(row[group_col]):03d}{int(row['particle_index']):02d}", axis=1
    ).astype(str)

    # Drop group_col from df2 before merging
    df2_repeated = df2_repeated.drop(columns=group_col)

    # Step 6: Concatenate
    merged_df = pd.concat([df1, df2_repeated], axis=1)

    return merged_df


In [17]:
split_columns = [
    'StreamDirectTN_kgday',
    'StreamDirectTP_kgday',
    'StreamDirectTN_gramday',
    'StreamDirectTP_gramday'
]

merged_df = expand_and_merge_by_group_id_with_split_loads(Initial_position, DirectNPcoast, split_cols=split_columns)


In [18]:
merged_df.describe()

,group_id,lon,lat,depth,particle_index,Shape_Area,StreamDirectTP_kgday,StreamDirectTN_kgday,StreamDirectTN_gramday,StreamDirectTP_gramday
count,8550.000000,8550.000000,8550.000000,8550.0,8550.000000,8.550000e+03,8550.000000,8550.000000,8550.000000,8550.000000
mean,77.061404,276.995615,43.866145,0.0,37.000000,3.075918e+08,0.209153,6.633769,6633.768937,209.152886
std,42.250960,0.576258,0.607709,0.0,21.649977,1.539575e+09,1.250936,36.219235,36219.235404,1250.935661
min,0.000000,275.330300,43.020300,0.0,0.000000,2.735322e+04,0.000043,0.000226,0.226400,0.042867
25%,42.000000,276.444400,43.491900,0.0,18.000000,4.967295e+06,0.004065,0.160095,160.094680,4.065039
50%,76.500000,277.384100,43.755100,0.0,37.000000,1.348578e+07,0.012966,0.571678,571.678022,12.965846
75%,116.000000,277.448300,43.983700,0.0,56.000000,6.282758e+07,0.047976,1.717673,1717.672931,47.975972
max,162.000000,277.591500,45.752400,0.0,74.000000,1.496975e+10,13.290446,383.363511,383363.510937,13290.446013


In [19]:
merged_df.tail()

,group_id,particle_id,lon,lat,depth,particle_index,Shape_Area,StreamDirectTP_kgday,StreamDirectTN_kgday,StreamDirectTN_gramday,StreamDirectTP_gramday
8545,162,16270,276.1465,43.6591,0.0,70,1.496975e+10,13.290446,383.363511,383363.510937,13290.446013
8546,162,16271,276.1477,43.6591,0.0,71,1.496975e+10,13.290446,383.363511,383363.510937,13290.446013
8547,162,16272,276.1489,43.6591,0.0,72,1.496975e+10,13.290446,383.363511,383363.510937,13290.446013
8548,162,16273,276.1501,43.6591,0.0,73,1.496975e+10,13.290446,383.363511,383363.510937,13290.446013
8549,162,16274,276.1513,43.6591,0.0,74,1.496975e+10,13.290446,383.363511,383363.510937,13290.446013


In [20]:
# Step 1: Count how many times each group_id occurs in df
group_counts = Initial_position['group_id'].value_counts().rename_axis('group_id').reset_index(name='count')
print(group_counts)

     group_id  count
0           0     75
1         116     75
2         113     75
3         112     75
4         111     75
..        ...    ...
109        50     75
110        49     75
111        48     75
112        45     75
113       162     75

[114 rows x 2 columns]


In [ ]:
# # drop the column 'StreamDirectTN_kgday' and 'StreamDirectTP_kgday'
merged_df = merged_df.drop(columns=['StreamDirectTN_kgday', 'StreamDirectTP_kgday'])
# rename Shape_Area column to 'Streamwatershed_Area'
merged_df = merged_df.rename(columns={'Shape_Area': 'Streamwatershed_Area'})
# save the merged DataFrame to a CSV file
output_file = os.path.join(out_path, 'initial_positions_NP.csv')
merged_df.to_csv(output_file,index=False)

# Adding NP load this to Particle trackking outputs based on group_id,particle_id

In [24]:
# Define the function to sort the files based on the time
def sort_key(file):
    filename = os.path.basename(file)
    try:
        # Extract the number after the double underscores and before the `.nc` extension
        number = int(filename.split('_')[-1].split('.')[0])
        return number
    except (IndexError, ValueError):
        # Handle filenames that do not match the pattern by returning a high number to place them last
        return float('inf')

In [36]:
# Set the directory of the FVCOM model outputs
FVCOM_dir = '/home/abolmaal/modelling/FVCOM/Huron/output/2024'
# read the updated NetCDF file
updated_files = glob.glob(FVCOM_dir + "/updated_FVCOM_Huron_2424_*.nc")
updated_files.sort(key=sort_key)

## Adding TN and TP tp particle tracking outputs 

✅ Remarks:

This function allows you to assign nutrient loads from stream watersheds to individual particles, ensuring that:

The load is divided fairly across the number of particles for each group.

The final output includes TN and TP flux and mass per particle.

The file is updated and saved with new nutrient tracking fields.

In [27]:
def update_nc_with_mass_flux(nc_file, merge_df, output_path):
    """
    Processes a single NetCDF particle tracking file:
    - Reads the file and extracts 'group_id' and 'particle_id'
    - Merges TN and TP load data per 'group_id' from `merge_df`
    - Assigns the same group-level values directly to each particle
    - Saves the updated NetCDF file with 'StreamTN_kgdayparticle' and 'StreamTP_kgdayparticle'
      (drops group-level columns before saving)
    """

    # Load the NetCDF file
    ds = xr.open_dataset(nc_file)

    # Convert time to datetime and count unique days
    time_vals = pd.to_datetime(ds['time'].values)
    unique_days = pd.to_datetime(time_vals).normalize().nunique()

    # Convert NetCDF to DataFrame
    df = ds[['group_id', 'particle_id']].to_dataframe().reset_index()

    # Merge in nutrient flux values
    flux_subset = merge_df[['group_id', 'StreamDirectTN_gramday', 'StreamDirectTP_gramday']].drop_duplicates()
    df = df.merge(flux_subset, on='group_id', how='inner')

    # Assign values directly (no division)
    
    df['StreamTN_gramdayparticle'] = df['StreamDirectTN_gramday']
    df['StreamTP_gramdayparticle'] = df['StreamDirectTP_gramday']

    # Sanity check
    assert len(df) == ds.sizes['particles'], "Mismatch between particle count in NetCDF and DataFrame."

    # Add only the per-particle values
    ds['StreamTN_gramdayparticle'] = (('particles'), df['StreamTN_gramdayparticle'].values)
    ds['StreamTP_gramdayparticle'] = (('particles'), df['StreamTP_gramdayparticle'].values)

    # Drop group-level columns before saving
    for var in ['StreamDirectTN_gramday', 'StreamDirectTP_gramday']:
        if var in ds.variables:
            ds = ds.drop_vars(var)

    # Save output
    filename = os.path.basename(nc_file)
    updated_file = os.path.join(output_path, f"particleload_{filename}")
    ds.to_netcdf(updated_file)
    print(f"✅ File saved: {updated_file}")

In [37]:
for nc in updated_files:  # list of your NetCDF files
    update_nc_with_mass_flux(nc, merged_df, FVCOM_dir)

✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_JanFeb_1.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_FebMar_2.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_MarApr_3.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_AprMay_4.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_MayJun_5.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_JunJul_6.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_JulAug_7.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_2424_AugSep_8.nc
✅ File saved: /home/abolmaal/modelling/FVCOM/Huron/output/2024/particleload_updated_FVCOM_Huron_

### Test

In [38]:
mass_updated = glob.glob(FVCOM_dir + "/particleload_updated_FVCOM_Huron_2424_*.nc")
mass_updated.sort(key=sort_key)


In [39]:
# Open the dataset
ds = xr.open_dataset(mass_updated[0])

In [40]:
ds

<xarray.Dataset> Size: 82MB
Dimensions:                   (time: 238, particles: 8550)
Coordinates:
  * time                      (time) datetime64[ns] 2kB 2024-01-01T18:00:00 ....
Dimensions without coordinates: particles
Data variables: (12/14)
    group_id                  (particles) int32 34kB ...
    longitude                 (time, particles) float32 8MB ...
    latitude                  (time, particles) float32 8MB ...
    depth                     (time, particles) float32 8MB ...
    host_fvcom                (time, particles) int32 8MB ...
    error_status              (time, particles) int32 8MB ...
    ...                        ...
    land_boundary_encounters  (time, particles) int32 8MB ...
    thetao                    (time, particles) float32 8MB ...
    so                        (time, particles) float32 8MB ...
    particle_id               (particles) <U9 308kB ...
    StreamTN_gramdayparticle  (particles) float64 68kB ...
    StreamTP_gramdayparticle  (particles) float64 68kB ...
Attributes:
    title:    PyLag -- Plymouth Marine Laboratory

In [41]:
# show the values of STreamTN_gramdayparticle and StreamTP_gramdayparticle for group_id= 0
group_id = 23
streamTN_values = ds['StreamTN_gramdayparticle'].where(ds['group_id'] == group_id, drop=True)
streamTP_values = ds['StreamTP_gramdayparticle'].where(ds['group_id'] == group_id, drop=True)
print(f"StreamTN values for group_id {group_id}:\n{streamTN_values}")
print(f"StreamTP values for group_id {group_id}:\n{streamTP_values}")

StreamTN values for group_id 23:
<xarray.DataArray 'StreamTN_gramdayparticle' (particles: 75)> Size: 600B
array([5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5152.27352284, 5152.27352284, 5152.27352284,
       5152.27352284, 5

In [33]:
# convert ds to pandas DataFrame 
ds_df = ds.to_dataframe().reset_index()
# count the number of occurrences where group_id == 0
group_id_counts = ds_df['group_id'].value_counts().reset_index()
print(f"Number of unique group_id where group_id== 0: {group_id_counts}")
# print the particle_id and StreamTN_gramdayparticle
print(ds_df[['group_id','particle_id','StreamTN_gramdayparticle', 'StreamTP_gramdayparticle']].head())


Number of unique group_id where group_id== 0:      group_id  count
0           0    300
1         116    300
2         113    300
3         112    300
4         111    300
..        ...    ...
109        50    300
110        49    300
111        48    300
112        45    300
113       162    300

[114 rows x 2 columns]
   group_id particle_id  StreamTN_gramdayparticle  StreamTP_gramdayparticle
0         0   000002301                179.744125                 12.995671
1         0   000012301                179.744125                 12.995671
2         0   000022301                179.744125                 12.995671
3         0   000032301                179.744125                 12.995671
4         0   000042301                179.744125                 12.995671


In [34]:
# Convert to DataFrame
df = ds[['group_id', 'particle_id', 'StreamTN_gramdayparticle', 'StreamTP_gramdayparticle']].to_dataframe().reset_index()

# Preview a few rows
print(df.head(10))

   particles  group_id particle_id  StreamTN_gramdayparticle  \
0          0         0   000002301                179.744125   
1          1         0   000012301                179.744125   
2          2         0   000022301                179.744125   
3          3         0   000032301                179.744125   
4          4         0   000042301                179.744125   
5          5         0   000052301                179.744125   
6          6         0   000062301                179.744125   
7          7         0   000072301                179.744125   
8          8         0   000082301                179.744125   
9          9         0   000092301                179.744125   

   StreamTP_gramdayparticle  
0                 12.995671  
1                 12.995671  
2                 12.995671  
3                 12.995671  
4                 12.995671  
5                 12.995671  
6                 12.995671  
7                 12.995671  
8                 12.9956

In [30]:
# save the DataFrame to a CSV file
output_file = os.path.join(out_path, 'mass_flux_data.csv')
df.to_csv(output_file, index=False)

In [35]:
# read the nc file
ds = xr.open_dataset(mass_updated[0])
# reat the variable massParticleTN
massParticleTN = ds['StreamTN_gramdayparticle'].values
print(massParticleTN)
# read the variable StreamDirectTN_kgday
#StreamDirectTN_kgday = ds['StreamDirectTN_gramday'].values
#StreamDirectTN_kgday


[1.79744125e+02 1.79744125e+02 1.79744125e+02 ... 3.83363511e+05
 3.83363511e+05 3.83363511e+05]


In [37]:
DirectNPcoast

,Group_id,Shape_Area,StreamDirectTP_kgday,StreamDirectTN_kgday,StreamDirectTN_gramday,StreamDirectTP_gramday
0,0,2.608193e+07,0.974675,13.480809,1.348081e+04,974.675289
1,1,6.915027e+07,2.000936,51.878556,5.187856e+04,2000.936025
2,2,1.703711e+08,4.179389,87.793043,8.779304e+04,4179.389247
3,3,9.425426e+07,3.598198,54.996792,5.499679e+04,3598.197912
4,4,3.772995e+07,1.326998,27.250568,2.725057e+04,1326.998050
...,...,...,...,...,...,...
140,170,1.961733e+07,4.299652,63.984811,6.398481e+04,4299.652109
141,171,1.228468e+03,0.000563,0.013980,1.398000e+01,0.563200
142,173,1.396735e+07,4.034916,23.636322,2.363632e+04,4034.915994
143,174,7.877722e+06,2.496025,14.078233,1.407823e+04,2496.024996
